In [1]:
import pathlib
from PIL import Image
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def show(data):
	plt.xticks([])
	plt.yticks([])
	plt.imshow(data)
	plt.show()

height, width = 2799, 2812
rowHeight = int(height / 15) - 1
colWidth = int(width / 15)
print(rowHeight, colWidth)

outputfilePath = pathlib.Path("dataTestProcessed")

img_height = 184
img_width = 187
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y',
               'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

185 187


# Load Model

In [2]:
TF_MODEL_FILE_PATH = "imageClassificationModel.tflite"
interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)
print(interpreter.get_signature_list())

classify_lite = interpreter.get_signature_runner('serving_default')

{'serving_default': {'inputs': ['rescaleImage_input'], 'outputs': ['output']}}


# Generate character matrix

In [3]:
def generateCharacterMatrix(pageNumber):
    print("Generating raw matrix")
    charMatrix = []
    x, y = 0, 0
    while x < 15:
        charMatrixRow = []
        y = 0
        while y < 15:
            imageName = "{}_{}_{}.jpg".format(pageNumber, x, y)
            imagePath = outputfilePath.joinpath(imageName)
            img = tf.keras.utils.load_img(str(imagePath), target_size=(img_height, img_width) )
            img_array = tf.keras.utils.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)

            predictions_lite = classify_lite(rescaleImage_input=img_array)['output']
            score_lite = tf.nn.softmax(predictions_lite)

            # print("Score: {} | Position: {} | Letter: {}".format(np.max(score_lite), np.argmax(score_lite), class_names[np.argmax(score_lite)]))
            charMatrixRow.append(class_names[np.argmax(score_lite)].upper())
            y += 1
        x += 1
        charMatrix.append(charMatrixRow)
    return np.array(charMatrix)

# Find Santa Routine

In [4]:
santas = ["ARUSHI", "CHANDAN", "MEGHANA", "SHRIDAR", "JERRY", "VAISHNAVI", "UTKARSH", "BRUNDA", "MOHAN", "SANDIL", "PARVATHY", "SIVA", "JUGNU", "HAMSINI", "ABUZAR", "KASHMEERA", "ABHIJEET", "DHEEMANTH", "NEELKANT", "ANURAAG", "NIRAV", "DEEPAK", "KAVI", "SNEHAL", "KARTHIK", "GAURAV", "NAVANEETA", "VIGNESH", "DEEPAK", "SUSHMITHA", "MOHNEESH", "YASH", "VIKAS", "GOPAKUMAR"]

def searchByRows(charImageMatrix):
	print("Searching rows")
	for i in range(15):
		line =""
		for c in charImageMatrix[i, :]:
			line += c
		for s in santas:
			count = line.count(s)
			if count > 0:
				return i, s
			count = line[::-1].count(s)
			if count > 0:
				return i, s
	return 0, ""

def searchByColumns(charImageMatrix):
	print("Searching columns")
	for i in range(15):
		line =""
		for c in charImageMatrix[:, i]:
			line += c
		for s in santas:
			count = line.count(s)
			if count > 0:
				return i, s
			count = line[::-1].count(s)
			if count > 0:
				return i, s
	return 0, ""

def searchDiagonally(charImageMatrix):
	print("Searching diagonally")
	max = 15
	diagonals = []
	for i in range(15):
		line = ""
		x = 0
		for loop in range(15):
			if x + i < 15:
				line += charImageMatrix[x][x+i]
			x += 1
		for s in santas:
			count = line.count(s)
			if count > 0:
				return x, i, s
			count = line[::-1].count(s)
			if count > 0:
				return x, i, s
	for i in range(15):
		i = i+1
		if(i == 15):
			break
		line = ""
		x = 0
		for loop in range(15):
			if x + i < 15:
				line += charImageMatrix[x+i][x]
			x += 1
		for s in santas:
			count = line.count(s)
			if count > 0:
				return x, i, s
			count = line[::-1].count(s)
			if count > 0:
				return x, i, s
	return 0, 0, ""


# Find Santa

> Start here

In [7]:
def findSanta(pageNumber):
	charImageMatrix = generateCharacterMatrix(pageNumber)
	print(charImageMatrix)
	row, santa = searchByRows(charImageMatrix)
	if santa != "":
		return santa
	col, santa = searchByColumns(charImageMatrix)
	if santa != "":
		return santa
	col, row, santa = searchDiagonally(charImageMatrix)
	if santa != "":
		return santa
	col, row, santa = searchDiagonally(charImageMatrix[::-1])
	return santa

# TEST-02 -> Jerry
# TEST-23 -> Vaishnavi
# TEST-34 -> Kashmeera

pageNumber = "TEST-34"
img = cv2.imread('pages/'+pageNumber+'.jpg')
# show(img)
print("-------- [START] - {} --------".format(pageNumber))
print("Santa -> {}".format(findSanta(pageNumber)))
print("-------- [DONE] - {} --------\n".format(pageNumber))

-------- [START] - TEST-34 --------
Generating raw matrix
[['M' 'S' 'U' 'E' 'Z' 'R' 'E' 'B' 'R' 'F' 'E' 'K' 'B' 'C' 'N']
 ['Y' 'F' 'M' 'K' 'W' 'J' 'I' 'A' 'V' 'M' 'T' 'E' 'M' 'U' 'X']
 ['S' 'X' 'A' 'C' 'N' 'X' 'M' 'R' 'F' 'M' 'C' 'O' 'V' 'W' 'D']
 ['G' 'V' 'S' 'I' 'W' 'U' 'Z' 'B' 'T' 'S' 'B' 'S' 'R' 'I' 'B']
 ['L' 'G' 'Q' 'H' 'K' 'D' 'I' 'Y' 'A' 'T' 'X' 'L' 'U' 'M' 'Z']
 ['Y' 'J' 'Y' 'A' 'B' 'K' 'O' 'J' 'L' 'O' 'S' 'M' 'W' 'G' 'Z']
 ['I' 'Z' 'P' 'F' 'G' 'I' 'R' 'S' 'F' 'Y' 'L' 'X' 'S' 'U' 'Q']
 ['A' 'A' 'Q' 'M' 'P' 'H' 'Q' 'S' 'O' 'K' 'G' 'M' 'K' 'X' 'X']
 ['G' 'E' 'T' 'A' 'D' 'B' 'Z' 'W' 'K' 'L' 'R' 'J' 'B' 'B' 'C']
 ['T' 'N' 'Q' 'J' 'M' 'Z' 'D' 'B' 'C' 'P' 'O' 'P' 'U' 'C' 'U']
 ['Y' 'O' 'H' 'G' 'M' 'Y' 'D' 'F' 'B' 'U' 'C' 'M' 'B' 'D' 'O']
 ['E' 'L' 'O' 'B' 'U' 'Q' 'P' 'Q' 'R' 'P' 'N' 'X' 'C' 'S' 'O']
 ['K' 'E' 'S' 'Y' 'Z' 'Y' 'A' 'A' 'L' 'K' 'C' 'P' 'X' 'R' 'G']
 ['D' 'D' 'G' 'P' 'F' 'M' 'E' 'Q' 'U' 'Y' 'K' 'D' 'D' 'H' 'M']
 ['J' 'X' 'Q' 'E' 'W' 'G' 'F' 'T' 'M' 'K' 'E' 'A' 'M' 'Q' 'B